In [1]:
import tensorflow as tf
import gzip
import matplotlib.pyplot as plt
import numpy as np
import os
from tqdm import tqdm_notebook as tqdm
import progressbar

/Users/ankit/anaconda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
/Users/ankit/anaconda/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [7]:
def create_tfrecord_file(images_file,labels_file, name):
    print("Processing output file {}".format(name))
    images = gzip.open(images_file)
    labels = gzip.open(labels_file)
    writer = tf.python_io.TFRecordWriter(name)
    images.seek(16)
    labels.seek(8)
    record_count = 0
    while True:
        data = images.read(784)
        if data == b'' or data == '':
            print('Wrote {} records'.format(record_count))
            writer.close()
            return
        
        label = int.from_bytes(labels.read(1), byteorder='little')
        image_feature = tf.train.Feature(
            bytes_list=tf.train.BytesList(value=[data])
        )
        label_feature = tf.train.Feature(
            int64_list=tf.train.Int64List(value=[label])
        )
        features_obj = tf.train.Features(
            feature={
                'label':label_feature,
                'image':image_feature
            }
        )
        example = tf.train.Example(features=features_obj)
        writer.write(example.SerializeToString())
        record_count += 1
        if record_count%10000 == 0:
            print(record_count)

        

In [8]:
create_tfrecord_file(
    images_file='emnist/emnist-balanced-train-images-idx3-ubyte.gz',
    labels_file='emnist/emnist-balanced-train-labels-idx1-ubyte.gz',
    name='train.tfrecord'
)
create_tfrecord_file(
    images_file='emnist/emnist-balanced-test-images-idx3-ubyte.gz',
    labels_file='emnist/emnist-balanced-test-labels-idx1-ubyte.gz',
    name='test.tfrecord'
)    

Processing output file train.tfrecord
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
Wrote 112800 records
Processing output file test.tfrecord
10000
Wrote 18800 records
